In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 10 17:34:18 2020

@author: Guillermo Sánchez Gutiérrez-Cabello
"""
#%%
import numpy as np
import pandas as pd
import os
import sklearn
#pd.options.mode.chained_assignment = None  # default='warn'


from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

import nltk
nltk.download("popular") # required to download the stopwords lists
from nltk.corpus import stopwords

spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')
#%%
mainpath = "C:/Users/guill/Documents/Universidad/PlataformaRefugiados/NAUTIA/DesarrolloPy/DataSetOriginales"

def fixBibliography(df):
    df = dfFix(df,"GENERAL INFORMATION - COUNTRY LEVEL")
    df.columns = ['GeneralInfo', 'CommunityCountry', 'RefugeeCountry']
    df.set_index('GeneralInfo', inplace = True)
    df = df.transpose()
    df = df.reset_index()
    return df

def dfFix(df,col1 = False,col2 = False):
    result = df.copy()
    if(col1):
        x = result.columns.get_loc(col1)
        result.drop(result.columns[0:x],axis = 1, inplace = True)
    if(col2):
        y = result.columns.get_loc(col2)
        result.drop(result.columns[y:],axis = 1, inplace = True)
    return result

def validColumn(cad):
    result = False
    if(cad == "index"):#Population
        result = True
    else:
        if(cad == "Type_of_settlement"):
            result = True
        else:
            if(cad == "General:settlement"):
                result = True
            else:
                if(cad == "general_info:_1_1_Choose_the_settlement"):
                    result = True
                else:
                    if(cad == "General_Information:Type_of_setlement"):
                        result = True
                    else:
                        if(cad == "General:Settlement"):
                            result = True
                        else:
                            if(cad == "Type_of_setlement"):
                                result = True
    return result

def dropRow(df,i):
    return df.drop(index = i)

def get_communityRows(df,cad,communityType): #la función pd.loc[] tiene un bug indiscriminado (https://github.com/pandas-dev/pandas/issues/8555)
    result = df
    if(communityType == 0):
        comm = "host_community"
    else:
        comm = "refugee_camp"
    for index, row in df.iterrows():
        if(row[cad] != comm):
            result = dropRow(result,index)
    return result

def setDataByIndex(df,communityType):
    array = df.columns
    i = 0
    df = df.replace("refugee","refugee_camp")
    df = df.replace("host_comunity","host_community")
    df = df.replace("RefugeeCountry","refugee_camp")
    df = df.replace("CommunityCountry","host_community")
    while(validColumn(array[i]) == False):
        i += 1
    df = get_communityRows(df,array[i],communityType)
    return df

def set_AllCSVtoDF(communityType):
    Bibliography = pd.read_excel(getPath(mainpath,"Bibliography_120220.xlsx"))
    Bibliography = fixBibliography(Bibliography)
    Bibliography = setDataByIndex(Bibliography,communityType)
    Entities = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Entities_Interview_results.csv")),communityType)
    LocalLeaders = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Local_leaders_v3_results.csv")),communityType)
    HouseHold = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Survey_household_v6_results.csv")),communityType)
    WomenGroup = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Women_Focus_Group2_results.csv")),communityType)
    SanitationInfra = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_V1_0_Sanitation_Infrastructre_results.csv")),communityType)
    Priorities = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Priorities_v3_results.csv")),communityType)
    GeneralForm = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_General_form_v3_results.csv")),communityType)
    PublicSpace = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Public_Space_results.csv")),communityType)
    WaterInf = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Water_Infrastructure_results.csv")),communityType)
    SanitationInf = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_V1_0_Sanitation_Infrastructre_results.csv")),communityType)
    WasteManagementInf = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Waste_Management_Infrastructure_results.csv")),communityType)
    EnergyINF = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Energy_Infrastructure_results.csv")),communityType)
    Business = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA1_0_Business_surveys_v3_results.csv")),communityType)
    MobilityINF = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0__Transport_servicesaccess_points_results.csv")),communityType) 
    ComunalServices = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Communal_Services_results.csv")),communityType) 
    GeneralCitizen = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_General_Citizen_Focus_Group_results.csv")),communityType)
    Shelter = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Shelter_results.csv")),communityType)
    FarmyardCrop = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Farmyard_and_Crops_results.csv")),communityType)
    return Bibliography,Entities,LocalLeaders,HouseHold,WomenGroup,SanitationInfra,Priorities,GeneralForm,PublicSpace,WaterInf,EnergyINF,SanitationInf,WasteManagementInf,EnergyINF,Business,MobilityINF,ComunalServices,GeneralCitizen,Shelter,FarmyardCrop

def concatDF(df1,df2):
    return  pd.concat([df1,df2],axis = 1, ignore_index = True, sort = True)

def mkCSV(df,fileName):
    df = df.dropna(how = 'all')
    df *= 1   
    fileName = fileName.lower()
    df.to_csv('DataSetFinales/'+fileName,sep=',',header = False, index=False, encoding='utf-8')
    
def getPath(mainpath,filename):
    return os.path.join(mainpath, filename)

def getSubColumnNames(df,x):
    columns = df.columns
    array = []
    for column in columns:
        column = column[x:]
        array.append(column)
    return pd.DataFrame(array) 

def addInstitutionAndType(df,array1,array2,instType):
    df = df.dropna(axis = 1)
    df = np.array(df)
    for row in df:
        for elem in row:
            array1 = np.append(array1,elem)
            array2 = np.append(array2,instType)
    return array1,array2

def politicalActor(df1,df2,df3,df4,df5):
    institution = []
    instType = []

    institution, instType  = addInstitutionAndType(df1,institution,instType,'Public Institution')
    institution, instType  = addInstitutionAndType(df2,institution,instType,'Private Institution')
    institution, instType  = addInstitutionAndType(df3,institution,instType,'NGO')
    institution, instType  = addInstitutionAndType(df4,institution,instType,'International Agency')
    institution, instType  = addInstitutionAndType(df5,institution,instType,'Local')

    institution = pd.DataFrame(institution)
    institution = institution.reset_index(drop = True)
    instType = pd.DataFrame(instType)
    instType = instType.reset_index(drop = True)
    
    return concatDF(institution,instType)


def get_claveValor(df1,df2):
    array1 = np.array(df2)
    array2 = np.array(df1)
    result1 = []
    result2 = []
    i = 0
    for row in array1:
        var = array2[i]
        for elem in row:
            result1 = np.append(result1,elem)
            result2 = np.append(result2,var)
        i+=1
    result2 = pd.DataFrame(result2)
    result2 = result2.reset_index(drop = True)
    result1 = pd.DataFrame(result1)
    result1 = result1.reset_index(drop = True)
    return concatDF(result2,result1)

def get_number(df):
    df = np.array(df)
    array = np.array([])
    for column in df:
        for elem in column:
            array = np.append(array,elem)
    return (pd.DataFrame(array)).fillna(0) 

def get_valueBySector(df1,df2):
    df2 = df2.reset_index()
    array1 = np.array(df1)
    i = 0
    for row in array1:
        for elem in row:
            if(elem == False):
                df2 = dropRow(df2,i)
        i += 1
    df2 = df2.set_index('index')
    return df2


def separateValues(df):
    if(df.isnull().values.all(axis=0)):
        result = df
    else:
        array = np.array(df)
        count_vectorizer = CountVectorizer(stop_words = spanish_stopwords+english_stopwords)
        corpus = []
        for row in array:
            for elem in row:
                corpus = np.append(corpus,[elem])
        X = count_vectorizer.fit_transform(corpus)
        array = count_vectorizer.get_feature_names()
        result = pd.DataFrame(array)
    return result

def vectorizeValue(df):
    df = separateValues(df)
    year = np.array(['january','february','march','april','may','june','july','august','september','october','november','december'])
    result = np.array([],dtype = bool)
    df = np.array(df)
    for elem in year:
        flag = False
        for column in df:
            for month in column:
                if(column == elem):
                    flag = True
        if(flag):
            result = np.append(result,True)
        else:
            result = np.append(result,False)
    return pd.DataFrame(result)

def set_sector(df,sect, concat = True):
    sector = np.array([])
    df = df.dropna(how = 'all')
    df = np.array(df)
    for column in df:
        sector = np.append(sector,sect)
    sector = pd.DataFrame(sector)
    df = pd.DataFrame(df)
    if(concat):
        result = concatDF(sector,df)
    else:
        result = sector
    return result 

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [2]:
Bibliography,Entities,LocalLeaders,HouseHold,WomenGroup,SanitationInfra,Priorities,GeneralForm,PublicSpace,WaterInf,EnergyINF,SanitationInf,WasteManagementInf,EnergyINF,Business,MobilityINF,ComunalServices,GeneralCitizen,Shelter,FarmyardCrop = set_AllCSVtoDF(1)

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
Bibliography.shape

(1, 192)

In [5]:
Entities.shape

(3, 72)

In [6]:
LocalLeaders.shape

(3, 22)

In [7]:
HouseHold.shape

(24, 42)

In [8]:
WomenGroup.shape

(1, 27)

In [9]:
SanitationInfra.shape

(37, 14)

In [10]:
Priorities.shape

(2, 88)

In [11]:
GeneralForm.shape

(1, 19)

In [12]:
PublicSpace.shape

(2, 14)

In [13]:
WaterInf.shape

(30, 13)

In [14]:
SanitationInf.shape

(37, 14)

In [15]:
WasteManagementInf.shape

(1, 10)

In [16]:
EnergyINF.shape

(4, 38)

In [17]:
Business.shape

(16, 25)

In [18]:
MobilityINF.shape

(3, 10)

In [19]:
ComunalServices.shape

(43, 63)

In [20]:
GeneralCitizen.shape

(1, 43)

In [21]:
Shelter.shape

(23, 27)

In [22]:
FarmyardCrop.shape

(16, 18)

In [23]:
PublicSpace.shape

(2, 14)

In [57]:
def dropNaAndResetIndex(df):
    df = df.dropna(how = 'all')
    df = np.array(df)
    return pd.DataFrame(df)
    

def get_applianceDF(df):
    df1 = dfFix(df,0,1)
    df2 = dfFix(df,1)
    array = np.array([])
    appliance = np.array([])
    for row in np.array(df1):
        corpus = np.array([])
        for elem in row:
            corpus = np.append(corpus,[elem])
        count_vectorizer = CountVectorizer(stop_words = spanish_stopwords+english_stopwords, vocabulary = ['light_bulbs','mobile_phone','radio_music_pl','tv_dvd','laptop_tablet_','fridge','electrical_sto','others'])
        X = count_vectorizer.fit_transform(corpus)
        array = count_vectorizer.get_feature_names()
        for elem in array:
            appliance = np.append(appliance,elem)
    appliance = pd.DataFrame(appliance)
    hours = np.array([])
    for row in np.array(df2):
        for elem in row:
            hours = np.append(hours,elem)
    hours = pd.DataFrame(hours)
    hours = (hours.dropna())
    return concatDF(appliance,hours)
    

In [69]:
df1 = dfFix(GeneralCitizen,"App_USED","App_needed")
df1 = set_sector(df1,"Used")
df2 = dfFix(GeneralCitizen,"App_needed","Type_Food:Meat")
df2 = set_sector(df2,"Necesity")
S_App_has_Community = concatDF(df1.T,df2.T).T
mkCSV(S_App_has_Community,"S_App_has_Community.csv")

In [70]:
S_App_has_Community

,0,1
0,Used,whatsapp
1,Necesity,whatsapp


In [68]:
GeneralCitizen

,start,end,Type_of_setlement,Type_of_Host_Community,Participant,Participant_001,Participant_002,Firewood_collection:Childs,Firewood_collection:Women,Firewood_collection:Men,Cooking:Childs_001,Cooking:Women_001,Cooking:Men_001,TICs_Knowledge:Phone_Call,TICs_Knowledge:Internet,TICs_Knowledge:PC,TICs_Knowledge:Programming,App_USED,App_needed,Type_Food:Meat,Type_Food:Grain,Type_Food:Vegetable,Type_Food:Fruit,times:One_time,times:Two_times,times:Three,times:More_than_Three,main_food:Breakfast,main_food:lunch,main_food:Coffe_time,main_food:Dinner,typical_dish:Pork,typical_dish:Vegetable_001,typical_dish:Beef,typical_dish:Chicken,typical_dish:LAMB,typical_dish:CEREALS,typical_dish:BEANS,typical_dish:FRUITS,Main_food_source:Humanitarian_Aid,Main_food_source:Their_owns_crops,Main_food_source:Market,meta:instanceID
0,2020-03-24 11:41:03.176,2020-03-24 12:32:31.531,refugee_camp,NaN,50.0,NaN,NaN,20.0,10.0,20.0,15.0,10.0,25.0,10.0,40.0,0.0,0.0,whatsapp,whatsapp,25.0,6.0,8.0,5.0,0.0,4.0,40.0,6.0,15.0,15.0,5.0,15.0,6.0,5.0,6.0,7.0,6.0,5.0,7.0,8.0,15.0,29.0,6.0,uuid:6d46fe01-2b52-4ec6-8021-f42840decfb4
